<a href="https://colab.research.google.com/github/arvyz/privacynlu/blob/main/qa_data_aug_looser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers --quiet
!pip install torch --quiet
!pip install accelerate --quiet
!pip install datasets --quiet
!pip install huggingface_hub --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.4 MB/s eta 0:00:00


In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
from transformers import AutoTokenizer, pipeline
import transformers
import torch
from huggingface_hub import HfApi

In [4]:
import pandas as pd
from datasets import Dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load Train & Test Data For Sentence Simplification

Training data from the privacy policies will be loaded and the privacy sentences will be "rewritten" for simplification.  Since each question in the QA set are mapped against all the sentences, there are many repeats of the same sentences across the dataset.  So we will only "rewrite" unique sentences and then map the newly generated senteces back to the QA set.

## Size
The full training corpus contains 3411 uniqe sentences (some sentences repeat across policies).

The full testing corpus is 1231 unique sentences (some sentences repeat across policies).

In [5]:
file_path = '/content/drive/My Drive/w266-personal/Final Project/PLUE/data/privacyqa/policy_train_data.csv'

# Load the CSV file
df_train = pd.read_csv(file_path, sep='\t')  # Adjust the path and separator based on your file

# Extract unique segments and create a Dataset
# We will also only look at unique sentences since they are repeated for each question.
unique_segments_df_full = pd.DataFrame({'Segment': df_train['Segment'].unique()})

In [6]:
file_path = '/content/drive/My Drive/w266-personal/Final Project/PLUE/data/privacyqa/policy_test_data.csv'

# Load the CSV file
df_test = pd.read_csv(file_path, sep='\t')  # Adjust the path and separator based on your file

# Extract unique segments and create a Dataset
# We will also only look at unique sentences since they are repeated for each question.
unique_segments_df_test = pd.DataFrame({'Segment': df_test['Segment'].unique()})

In [7]:
display(unique_segments_df_full.shape)
display(unique_segments_df_full.head())
display(unique_segments_df_test.shape)
display(unique_segments_df_test.head())

(3411, 1)

,Segment
0,"This privacy policy, with our Terms of Servic..."
1,We encourage you to read this privacy policy c...
2,By using our application or other online servi...
3,"When we post changes to this privacy policy, ..."
4,We encourage you to review this privacy policy...


(1231, 1)

,Segment
0,At Fiverr we care about your privacy.
1,We do not sell or rent your personal informati...
2,We do not disclose it to others except as disc...
3,We collect information that you provide us o...
4,We will not share information that you provi...


## Sampling Process for getting heuristic parameters
We will look at a smaller "sample" of sentences (n=200) to defined the heuristic weights for the "rewriting" process.  The smaller set is sampled across all the polcies to get a wide variety of sentences etc to be able to valdate the data prep process and appropriate prompts for the final "rewriting"

In [8]:
# Let's first experiment on a few 100 sentences
sample = 200
unique_segments_df_sample = unique_segments_df_full.sample(n=sample)
display(unique_segments_df_sample.shape)
display(unique_segments_df_sample.head())

(200, 1)

,Segment
480,Any information that you post in these locatio...
2912,Provide personalized experiences
2833,When you provide your information to Epic as p...
604,Our name is an acronym for the Latin p...
88,"If you do not agree to the changes, you should..."


# Prompt Engineering

To simplify each of the sentences of the privacy policy, we will use a Large Language Model (LLM) to generate the ouput.  The model we will be using is Meta's LLAMA2 7B model that has been fine tuned for "Chat" contexts to be able to take a chat prompt and produce a reponses.  

## Prompt Attempts
LLMs are sensitve to prompts for generating appropriate text.  So it is important to test out various prompts to the drive the right type of reponse from the LLM.

The following prompts were tested.  The final prompt chosed in bolded:

1.   rephrase and simplify this sentence for clarity for an average user:
2.   Rephrase and simplify the following sentence for clarity for an average user and do not add any qualifiying statements to the response.
3.   **Rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE:**
4.  Rephrase and simplify for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE:
5. Rephrase, simplify, and clarify for an average user; do not add any qualifiying statements to the response.  SENTENCE:

A few additional other prompts were tested, but they did not work at all.  So I have excluded it from the data

## Prompt Selection Process

Sample "rewritten" data was generated for all the above prompts and the final prompt for the QA experiment was selected based on visual inspection of results to ensure the generated text reasonably and faithfully represented the underlying intent of the sentence.  ***NOTE:*** This was not done by a privacy experty by the Research principle in consultation with another lay-person.  I had to balance between some really great simplifications, but some really awkward hallucinations  See below for two different sets of such responses.



|                              | Sample Prompt & Responses 1                          | Sample Prompt & Responses 2                           |
|------------------------------|------------------------------------------------------|-------------------------------------------------------|
| **Original Sentence:**      | When You Give Consent                                | Web Behavior Data                                     |
|                              | We may share information about you with other       |                                                       |
|                              | companies if you give us permission or direct us   |                                                       |
|                              | to share the information.                            |                                                       |
| **Prompt 1 Response:**      | Answer: When you give permission, we may share      | *There was no response*                               |
|                              | information about you with other companies.         |                                                       |
| **Prompt 3 Response:**      | REPHRASED SENTENCE: We may share information       | Rephrased and simplified sentence: Data on how        |
|                              | about you with other companies if you give us       | people use the web.                                   |
|                              | permission or direct us to do so.                   |                                                       |
| **Prompt 4 Response:**      | REPHRASED SENTENCE: When you give permission,      | Web Behavior Data refers to the information          |
|                              | we may share information about you with other      | collected about a user's online activities,          |
|                              | companies.                                           | including their browsing history, search queries,    |
|                              |                                                      | and interactions with websites and apps. This data   |
|                              |                                                      | can be used to create a detailed profile of the     |
|                              |                                                      | user's online behavior, which can be used to        |
|                              |                                                      | personalize their online experience, improve the    |
|                              |                                                      | effectiveness of online advertising, and enhance    |
|                              |                                                      | the overall user experience.                         |
| **Prompt 5 Response:**      | REPHRASED SENTENCE: If you agree, we can share     | Web Behavior Data refers to the information          |
|                              | information about you with other companies.         | collected about a user's online activities,          |
|                              |                                                      | including their browsing history, search queries,    |
|                              |                                                      | and interactions with websites and applications.    |
|                              |                                                      | This data can be used to create a detailed profile |
|                              |                                                      | of the user's online behavior, which can be used to|
|                              |                                                      | personalize their online experience, improve the    |
|                              |                                                      | effectiveness of online advertising, and enhance    |
|                              |                                                      | the overall user experience.                         |


In [9]:
# Function to add prompts to each segment
def add_prompts_4(row):
    prompt = f"Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE: {row['Segment']}\n"
    return {"Prompt": prompt}

# Let's use a common variable name to help with code replication
dataset_4 = Dataset.from_pandas(unique_segments_df_sample)
# Apply function to create prompts
dataset_4 = dataset_4.map(add_prompts_4)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [10]:
# Based on Trail 4; Simple prompt; Higher Temperature

# Function to add prompts to each segment
def add_prompts_full(row):
    prompt = f"Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE: {row['Segment']}\n"
    return {"Prompt": prompt}

# Let's use a common variable name to help with code replication
dataset_full = Dataset.from_pandas(unique_segments_df_full)
# Apply function to create prompts
dataset_full = dataset_full.map(add_prompts_full)

Map:   0%|          | 0/3411 [00:00<?, ? examples/s]

In [11]:
# Based on Trail 4; Simple prompt; Higher Temperature
# Let's do the same treatment for the Test sentences as we did for the training sentences
dataset_full_test = Dataset.from_pandas(unique_segments_df_test)

# Apply function to create prompts
dataset_full_test = dataset_full_test.map(add_prompts_full)

Map:   0%|          | 0/1231 [00:00<?, ? examples/s]

In [12]:
print("Dataset 4: ", dataset_4[0]['Prompt'])
print("Dataset Full Train: ", dataset_full[0]['Prompt'])
print("Dataset Full Test: ", dataset_full_test[0]['Prompt'])

Dataset 4:  Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE: Any information that you post in these locations, or in our user forum, can be read, collected, and used by anyone and could enable others to send you unsolicited messages.

Dataset Full Train:  Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE:  This privacy policy, with our Terms of Service & License Agreement, is the agreement between you and us that explains what data we collect, how we store and transfer it, and how we use it.

Dataset Full Test:  Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE:   At Fiverr we care about your privacy.



In [13]:
#!pip install accelerate

# Initialize the tokenizer and the model pipeline
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
gen_pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    tokenizer=tokenizer,
    torch_dtype=torch.float32,
    device_map="auto",
    framework="pt"
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [14]:
# Function to generate text using the pipeline
def generate_text4(batch):
    # Process batch of prompts through the pipeline
    outputs_4 = gen_pipeline(batch['Prompt'], do_sample=True,
                           temperature=0.6, top_p=0.7, top_k=10, num_beams=3, early_stopping=False,
                           num_return_sequences=1, eos_token_id=tokenizer.eos_token_id,
                           max_length=200, truncation=True)
    # Extract the generated text and clean it up by removing the prompt text
    return {"Generated_Text": [output[0]['generated_text'].split('\n')[-1].strip() for output in outputs_4]}

# Apply the function batch-wise to the dataset
result_4 = dataset_4.map(generate_text4, batched=True, batch_size=20)

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [21]:
ver = 4
result_4.shape
for i in range(result_4.num_rows):
  print("Sentence:",  result_4[i]['Segment'])
  print("Prompt:", result_4[i]['Prompt'])
  print("New Sentence:", result_4[i]['Generated_Text'])
  print()

import pandas as pd

# write out to file
output_file_path = f'/content/drive/My Drive/w266-personal/Final Project/PLUE/data/privacyqa/policy_train_dataset_small_loose_{sample}_{ver}.csv'

# Save to CSV
df_4 = result_4.to_pandas()
df_4.to_csv(output_file_path, index=False)

Sentence: Any information that you post in these locations, or in our user forum, can be read, collected, and used by anyone and could enable others to send you unsolicited messages.
Prompt: Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE: Any information that you post in these locations, or in our user forum, can be read, collected, and used by anyone and could enable others to send you unsolicited messages.

New Sentence: REPHRASED SENTENCE: Anyone can read and collect information you post in certain locations, including our user forum, and use it to send you unsolicited messages.

Sentence: Provide personalized experiences
Prompt: Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE: Provide personalized experiences

New Sentence: REPHRASED SENTENCE: Offer tailored experiences based on use

## Hyperparameter Tuning

To decide the best hyper-parameters for Sentence generation for *simplifying, clarifying, and rephrasing the original sentence succinctly* I try out multiple variations of hyperparameters that can be used to control the language generation process.  These include:

*   temperature - Default, 0.25, 0.35, 0.4, 0.5, 0.6
*   top_p - Default, 0.8, 0.6, 0.7,
*   top_k - 5, 7, 10
*   num_beams - Default, 2, 3
*   early_stopping - True, False

## Chosen Values for Sentence rewriting (Version 2)

 * temperature=0.6,
 * top_p=0.7,
 * top_k=10,
 * num_beams=3,
 * early_stopping=False,



In [15]:
def get_segment(dataset, prompt):
    """
    Retrieve the segment associated with a specific prompt from a Hugging Face Dataset.

    Parameters:
    - dataset (Dataset): A Hugging Face Dataset object with columns ['Prompt', 'Segment', 'Generated_Text'].
    - prompt (str): The prompt string to search for within the dataset.

    Returns:
    - str: The segment corresponding to the given prompt. Returns 'No segment found' if the prompt does not exist.
    """
    # Filter the dataset for the target prompt
    filtered_dataset = dataset.filter(lambda example: example['Prompt'] == prompt)

    # Extract the segment for the specific prompt
    if len(filtered_dataset) > 0:
        return filtered_dataset['Segment'][0]
    else:
        return "No segment found"

In [16]:
import json
from datasets import concatenate_datasets, load_dataset

# Based on Trail #2; Simple prompt with SENTENCE Marker
# Low temperature; Try to flex on Top_p and Beams.
# Function to generate text using the pipeline
def generate_text_full(batch):

    # Initialize the batch counter attribute if it doesn't exist
    if not hasattr(generate_text_full, 'batch_counter'):
        generate_text_full.batch_counter = 0  # Static variable equivalent
    generate_text_full.batch_counter += 1  # Increment the counter
    print(f"Processing Batch #{generate_text_full.batch_counter}, Batch Size: {len(batch['Prompt'])}")

    try:
        # Process batch of prompts through the pipeline with torch.no_grad() for memory efficiency
        with torch.no_grad():
            outputs_full = gen_pipeline(batch['Prompt'], do_sample=True,
                           temperature=0.6, top_p=0.7, top_k=10, num_beams=3, early_stopping=False,
                           num_return_sequences=1, eos_token_id=tokenizer.eos_token_id,
                           max_length=200, truncation=True)
        # Extract the generated text and clean it up by removing the prompt text
        return {"Generated_Text": [output[0]['generated_text'].split('\n')[-1].strip() for output in outputs_full]}
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Reverting to sequential processing for this batch")
        generated_texts = []
        for prompt in batch['Prompt']:
          try:
            output = gen_pipeline(prompt, do_sample=True,
                           temperature=0.6, top_p=0.7, top_k=10, num_beams=3, early_stopping=False,
                           num_return_sequences=1, eos_token_id=tokenizer.eos_token_id,
                           max_length=200, truncation=True)
            # Extract the generated text and clean it up by removing the prompt text
            generated_text = output[0]['generated_text'].split('\n')[-1].strip()

            # Append the cleaned generated text to the list
            generated_texts.append(generated_text)

          except Exception as ex:
            print(f"Error processing prompt: {prompt}.")
            print(f"Using source segment.")
            print(f"Error: {ex}")
            combined_dataset = concatenate_datasets([dataset_full, dataset_full_test])
            generated_texts.append(get_segment(combined_dataset,prompt))  # Use the original segment in case of an error

        # Return the list of generated texts
        return {"Generated_Text": generated_texts}

In [17]:
# Apply the function batch-wise to the dataset
result_full = dataset_full.map(generate_text_full, batched=True, batch_size=100)

Map:   0%|          | 0/3411 [00:00<?, ? examples/s]

Processing Batch #1, Batch Size: 100


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Processing Batch #2, Batch Size: 100
Processing Batch #3, Batch Size: 100
Processing Batch #4, Batch Size: 100
Processing Batch #5, Batch Size: 100
Processing Batch #6, Batch Size: 100
Processing Batch #7, Batch Size: 100
Processing Batch #8, Batch Size: 100
An error occurred: Input length of input_ids is 200, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.
Reverting to sequential processing for this batch
Error processing prompt: Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE:  Except as otherwise described in this privacy statement, Goodreads will not disclose personal information to any third party unless required to do so by law or subpoena or if we believe that such action is necessary to (a) conform to the law, comply with legal process served on us or our affiliates, or in

Filter:   0%|          | 0/4642 [00:00<?, ? examples/s]

...

Error processing prompt: Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE: The Data subject has the right to: a) obtain the termination of processing for direct marketing purposes, also in relation   to services identical to those already provided by our company; b) obtain information in relation to the purposes for which   Data are processed, to the period of processing and to the subjects to whom the data are communicated (right of access);   c) obtain the rectification or integration of incorrect Data concerning the Data subject (right to rectification); d) obtain   the deletion of Data concerning the Data subject in the following cases (i) the data are no longer necessary for the purposes   for which they were collected; (ii) the Data subject has withdrawn his/her consent to the processing of Data, if these   are processed on the basis of his consent; (iii) the Data subject has oppos

Filter:   0%|          | 0/4642 [00:00<?, ? examples/s]

Error processing prompt: Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE: However,   it is to be noted that the retention of Data by Inter is also lawful if it is necessary in order to fulfil a legal obligation   or to ascertain, exercise or defend a right in Court (cancellation right); b) to obtain that the Data concerning the Data   subject are only stored without any further use in the following cases: (i) the Data subject disputes the accuracy of the   Data, for the period that is necessary for allowing Inter to verify the accuracy of such Data; (ii) the processing is unlawful   but the Data subject objects to the deletion of the Data; (iii) the Data are necessary for ascertaining, exercising or   defending a right in Court; (iv) the Data subject has opposed the processing and is awaiting the verification of the possible   prevalence of Inters legitimate interest to the processing (

Filter:   0%|          | 0/4642 [00:00<?, ? examples/s]

Processing Batch #25, Batch Size: 100
Processing Batch #26, Batch Size: 100
Processing Batch #27, Batch Size: 100
Processing Batch #28, Batch Size: 100
Processing Batch #29, Batch Size: 100
Processing Batch #30, Batch Size: 100
Processing Batch #31, Batch Size: 100
Processing Batch #32, Batch Size: 100
Processing Batch #33, Batch Size: 100
Processing Batch #34, Batch Size: 100
Processing Batch #35, Batch Size: 11


## Save Unique Values to File

Print out a small sample for visual inspection. Wrtie the full set out to file.

In [18]:
import pandas as pd

# write out to file
output_file_path = f'/content/drive/My Drive/w266-personal/Final Project/PLUE/data/privacyqa/policy_train_dataset_full_loose.csv'

# Save to CSV
df_full = result_full.to_pandas()
df_full.to_csv(output_file_path, index=False)

# Test Set "rewriting" Simplification

Let's rewrite the test QA sentences using the same LLM text generation process

In [19]:
# Apply the function batch-wise to the dataset
result_full_test = dataset_full_test.map(generate_text_full, batched=True, batch_size=100)

Map:   0%|          | 0/1231 [00:00<?, ? examples/s]

Processing Batch #36, Batch Size: 100


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Processing Batch #37, Batch Size: 100
Processing Batch #38, Batch Size: 100
Processing Batch #39, Batch Size: 100
An error occurred: Input length of input_ids is 200, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.
Reverting to sequential processing for this batch
Error processing prompt: Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE: with relevantBusiness Partners:with whom we jointly offer products and services;with whom we have entered into an agreement that provides for the disclosure, sale, lease, or license of your Information;to facilitate a direct relationship with you, including in connection with any program we administer on behalf of theBusiness Partner;to enable electronic communications with you as part of purchase, a sponsored reward, offer, contest, program, or ot

Filter:   0%|          | 0/4642 [00:00<?, ? examples/s]

...

Processing Batch #45, Batch Size: 100
An error occurred: Input length of input_ids is 200, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.
Reverting to sequential processing for this batch
Error processing prompt: Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE: 23andMe will preserve and disclose any and all information to law enforcement agencies or others if required to do so by law or in the good faith belief that such preservation or disclosure is reasonably necessary to: (a) comply with legal or regulatory process (such as a judicial proceeding, court order, or government inquiry) or obligations that 23andMe may owe pursuant to ethical and other professional rules, laws, and regulations; (b) enforce the 23andMe Terms of Service and other policies; (c) respond to claims that a

Filter:   0%|          | 0/4642 [00:00<?, ? examples/s]

Processing Batch #46, Batch Size: 100
Processing Batch #47, Batch Size: 100
An error occurred: Input length of input_ids is 200, but `max_length` is set to 200. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.
Reverting to sequential processing for this batch
Error processing prompt: Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE: (a)Make our service available: We use your Registration and Account information to (i) register you for the App and create your User Account for Viber; (ii) create your profile and make it visible; (iii) process your payments; (iv) create your Viber ID; (v) provide customer service, give you information about your account, and respond to your requests; (vi) personalize your experience by providing content (such as games) on the Service, including targeted advertising of Viber services 

Filter:   0%|          | 0/4642 [00:00<?, ? examples/s]

Processing Batch #48, Batch Size: 31


## Save Unique Values to File

Print out a small sample for visual inspection. Wrtie the full set out to file.

In [20]:
sample = 50
ver = "full_test"
result_full_test.shape
for i in range(sample):
  print("Sentence:",  result_full_test[i]['Segment'])
  print("Prompt:", result_full_test[i]['Prompt'])
  print("New Sentence:", result_full_test[i]['Generated_Text'])
  print()

import pandas as pd

# write out to file
output_file_path = f'/content/drive/My Drive/w266-personal/Final Project/PLUE/data/privacyqa/policy_test_dataset_full_loose.csv'

# Save to CSV
df_full_test = result_full_test.to_pandas()
df_full_test.to_csv(output_file_path, index=False)

Sentence:   At Fiverr we care about your privacy.
Prompt: Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE:   At Fiverr we care about your privacy.

New Sentence: REPHRASED SENTENCE:  Fiverr cares about your privacy.

Sentence: We do not sell or rent your personal information to third parties for their direct marketing purposes without your explicit consent.
Prompt: Succinctly rephrase and simplify the 'SENTENCE' for clarity for an average user and do not add any qualifiying statements to the response.  SENTENCE: We do not sell or rent your personal information to third parties for their direct marketing purposes without your explicit consent.

New Sentence: REPHRASED SENTENCE: We don't share your personal information with third parties for their marketing purposes without your consent.

Sentence: We do not disclose it to others except as disclosed in this Policy or required to provide y

# Save Unique Values to File for Trail Attempts

All the various Trial attemps from 1 -> 6
Print out a small sample for visual inspection. Wrtie the full set out to file.

# Clean up and format output for our modeling files
The output, upon inspection, has multiple types of labels that precede sentences like:

* "REPHRASED SENTENCE:",
* "ANALYSIS:",
* "CLEAR AND CLEAR SENTENCE:",
* etc.

We want to extract the sentences without these labels.  We'll do this with Regular expressions (regex).

In [22]:
def clean_generated_text(example):
    # Define the list of specific prefixes to remove
    prefixes = [
        "REPHRASED SENTENCE", "ANONYMIZED =", "SIMPLIFIED SENTENCE",
        "Answer", "CORRECTED SENTENCE", "Rephrased and simplified sentence",
        "Simplified Rephrased Sentence", "Rephrased and simplified",
"Simplified Response",
"Simplified",
"Simplified and rephrased",
"SUCCINCT REPHRASE",
"Rephrased and simplified",
"The simplified and rephrased sentence is",
"Simplified and rephrased sentence",
"ANONYMIZATION",
"Simplified Rephrased Response",
"SUCCINCT REPHRASING",
"CORRECTED SENTENCE",
"Rephrased and simplified sentence",
"Rephrased",
"The sentence \"The company's new policy requires all employees to use their personal laptops for work-related tasks by the end of the year.\" can be rephrased and simplified as",
"Revised SENTENCE",
"Retained responses",
"Simpliifed Response",
"The simplified sentence is",
"The sentence can be rephrased as",
"The simplified rephrased sentence is",
"The sentence is rephrased and simplified as",
"New services =",
"The sentence can be rephrased and simplified as",
"RENDERED SENTENCE",
"The 'SENTENCE' can be rephrased as",
"Simplified explanation",
"The sentence is rephrased as",
"Rephrase and simplify the sentence for an average user.",
"ACCESSIBLE VERSION",
"SUCCINCTLY REPHRASED SENTENCE",
"SIMPLIFIED REPHRASE",
"Modified sentence",
"NICKNAME",
"The sentence \"if you enter into a website, it recognizes you during your next visit\" can be rephrased and simplified as",
"Simplified response",
"Simplified Rephrased Sentence",
"Rephrased and simplified sentence",
"SUCCINCT REPHRASE",
"SUMMARIZED SENTENCE",
"REPHRASED SENTEN",
"Simpliified response"
    ]

    # Prepare regex to match any of these prefixes at the start of a string followed by possible colon and spaces
    import re
    pattern = re.compile(r'^(?:' + '|'.join(re.escape(prefix) for prefix in prefixes) + r')\s*[:=]?\s*')

    # Function to clean each text in the list
    def clean_text(text):
        # Apply regex to remove the prefix and strip whitespace
        return pattern.sub('', text).strip()

    # Apply the cleaning function to each text in the list 'Generated_Text'
    cleaned_texts = [clean_text(text) for text in example['Generated_Text']]

    return {'Clean_Gen_Segment': cleaned_texts}

In [23]:
# 'result_full' is the Dataset object from the previous text generation step
cleaned_results_train = result_full.map(clean_generated_text, batched=True)
cleaned_results_test = result_full_test.map(clean_generated_text, batched=True)

Map:   0%|          | 0/3411 [00:00<?, ? examples/s]

Map:   0%|          | 0/1231 [00:00<?, ? examples/s]

In [24]:
cleaned_results_train.to_pandas().head()

,Segment,Prompt,Generated_Text,Clean_Gen_Segment
0,"This privacy policy, with our Terms of Servic...",Succinctly rephrase and simplify the 'SENTENCE...,REPHRASED SENTENCE: This policy explains what...,"This policy explains what data we collect, how..."
1,We encourage you to read this privacy policy c...,Succinctly rephrase and simplify the 'SENTENCE...,REPHRASED SENTENCE: Please read our privacy po...,Please read our privacy policy when using our ...
2,By using our application or other online servi...,Succinctly rephrase and simplify the 'SENTENCE...,REPHRASED SENTENCE: By using our app or other ...,"By using our app or other online services, you..."
3,"When we post changes to this privacy policy, ...",Succinctly rephrase and simplify the 'SENTENCE...,REPHRASED SENTENCE: We update this privacy po...,We update this privacy policy when changes are...
4,We encourage you to review this privacy policy...,Succinctly rephrase and simplify the 'SENTENCE...,REPHRASED SENTENCE: Please review our privacy ...,Please review our privacy policy regularly.


In [25]:
# Convert results to DataFrame
results_df_train = cleaned_results_train.to_pandas()

# Create a dictionary to map original segments to their simplified versions
simplified_map = dict(zip(results_df_train['Segment'], results_df_train['Clean_Gen_Segment']))

# Map the simplified texts back to the original dataframe
df_train['Simplified_Segment'] = df_train['Segment'].map(simplified_map)

# Define the path to save the modified file
output_path_train = '/content/drive/My Drive/w266-personal/Final Project/PLUE/data/privacyqa/policy_train_data_modified_loose.csv'

# Save the DataFrame back to a CSV file
df_train.to_csv(output_path_train, sep='\t', index=False)

In [26]:
# Convert results to DataFrame
results_df_test = cleaned_results_test.to_pandas()

# Create a dictionary to map original segments to their simplified versions
simplified_map_test = dict(zip(results_df_test['Segment'], results_df_test['Clean_Gen_Segment']))

# Map the simplified texts back to the original dataframe
df_test['Simplified_Segment'] = df_test['Segment'].map(simplified_map_test)

# Define the path to save the modified file
output_path_test = '/content/drive/My Drive/w266-personal/Final Project/PLUE/data/privacyqa/policy_test_data_modified_loose.csv'

# Save the DataFrame back to a CSV file
df_test.to_csv(output_path_test, sep='\t', index=False)